In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import os

In [2]:
import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
import pandas as pd

from copy import deepcopy

In [3]:
word_vec = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True, unicode_errors='ignore')

In [5]:
df = pd.read_csv("data_final.csv")
df.head(5)

,제품 번호,제품명,Top Note,Middle Note,Base Note,브랜드,부향률,가격,설명
0,0,겐조 플라워 바이 겐조 오 드 퍼퓸,"블랙커런트, 산사나무, 불가리안 로즈","오포포낙스, 자스민, 파라마 바이올렛, 로즈, 머스크","바닐라, 인센스",겐조,오 드 퍼퓸,"106,000",NaN
1,1,겐조 뿌르팜므 EDT,"만다린, 그린 블랙커런트",라즈베리 잎사귀,매그놀리아 꽃잎,겐조,오 드 뚜왈렛,"45,000",NaN
2,2,겐조 우먼 EDT,"라일락, 만다린, 워터민트","아마릴리스, 수련, 복숭아","머스크, 삼나무, 바닐라",겐조,오 드 뚜왈렛,"50,900",NaN
3,3,겐조 뉴 플라워 바이 겐조 EDT,시칠리아 레몬,불가리아 로즈 워터,"머스크, 바닐라",겐조,오 드 뚜왈렛,"64,000",NaN
4,4,겔랑 아쿠아 알레고리아 페라 그라니타 오 드 뚜왈렛,"자몽, 레몬, 베르가못","배, 오렌지 블로썸, 오스만투스, 헤디온","머스크, 세다, 모스",겔랑,오 드 뚜왈렛,"158,000",NaN


In [8]:
''' 설명
top_dict : raw data
top_dict_new : 임의의 단어로 대체한 dict'''

top_dict, middle_dict, base_dict = {}, {}, {}

with open("top_dict_new", "r") as f:
    for line in f.readlines():
        if line:
            v = line.strip().split(", ")
            top_dict[v[0]] = v[1].lower().replace("*", "").split()


with open("middle_dict_new", "r") as f:
    for line in f.readlines():
        if line:
            v = line.strip().split(", ")
            try:
                middle_dict[v[0]] = v[1].lower().replace("*", "").split()
            except:
                print(v)
                raise 'o'

with open("base_dict_new", "r") as f:
    for line in f.readlines():
        if line:
            v = line.strip().split(", ")
            base_dict[v[0]] = v[1].lower().replace("*", "").split()

In [9]:
# word_vec에 없는 단어 체크
top_vec = {}

for k, l in top_dict.items():
    vecs = None
    for v in l:
        try:
            if vecs is None:
                vecs = deepcopy(word_vec[v])
            else:
                vecs += deepcopy(word_vec[v])
        except:
            continue
    if vecs is not None:
        top_vec[k] = vecs
    else:
        top_vec[k] = np.zeros(300)
        print("벡터값 0!", k, l)

In [10]:
middle_vec = {}

for k, l in middle_dict.items():
    vecs = None
    for v in l:
        try:
            if vecs is None:
                vecs = deepcopy(word_vec[v])
            else:
                vecs += deepcopy(word_vec[v])
        except:
            continue
    if vecs is not None:
        middle_vec[k] = vecs
    else:
        middle_vec[k] = np.zeros(300)
        print("벡터값 0!", k, l)

In [11]:
base_vec = {}

for k, l in base_dict.items():
    vecs = None
    for v in l:
        try:
            if vecs is None:
                vecs = deepcopy(word_vec[v])
            else:
                vecs += deepcopy(word_vec[v])
        except:
            continue
    if vecs is not None:
        base_vec[k] = vecs
    else:
        base_vec[k] = np.zeros(300)
        print("벡터값 0!", k, l)

In [12]:
top_lambda = lambda k: top_vec[k]
df["Top Note Vec"] = df["Top Note"].apply(lambda s: s.split(", ")).apply(lambda l: [i.strip() for i in l]).apply(lambda l: np.sum(list(map(top_lambda, l)), axis=0))

middle_lambda = lambda k: middle_vec[k]
df["Middle Note Vec"] = df["Middle Note"].apply(lambda s: s.split(", ")).apply(lambda l: [i.strip() for i in l]).apply(lambda l: np.sum(list(map(middle_lambda, l)), axis=0))

base_lambda = lambda k: base_vec[k]
df["Base Note Vec"] = df["Base Note"].apply(lambda s: s.split(", ")).apply(lambda l: [i.strip() for i in l]).apply(lambda l: np.sum(list(map(base_lambda, l)), axis=0))


In [13]:
# note 앙상블
df["Ensemble Vec"] = df['Top Note Vec'] * 0.1 + df['Middle Note Vec'] * 0.1 + df['Base Note Vec'] * 0.1

In [14]:
# 모든 note 합치기
df["Total Vec"] = (df['Top Note Vec'].apply(lambda x: x.tolist()) + df['Middle Note Vec'].apply(lambda x: x.tolist()) + df['Base Note Vec'].apply(lambda x: x.tolist())).apply(np.array)

In [15]:
top_vecs = np.array(df["Top Note Vec"].tolist())
middle_vecs = np.array(df["Middle Note Vec"].tolist())
base_vecs = np.array(df["Base Note Vec"].tolist())
total_vecs = np.array(df["Total Vec"].tolist())
ensemble_vecs = np.array(df["Ensemble Vec"].tolist())

In [16]:
# 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity
top_nn = cosine_similarity(top_vecs, top_vecs).argsort(axis=1)[:,::-1][:, 1:10]
middle_nn = cosine_similarity(middle_vecs, middle_vecs).argsort(axis=1)[:,::-1][:, 1:10]
base_nn = cosine_similarity(base_vecs, base_vecs).argsort(axis=1)[:,::-1][:, 1:10]
total_nn = cosine_similarity(total_vecs, total_vecs).argsort(axis=1)[:,::-1][:, 1:10]
ensemble_nn = cosine_similarity(ensemble_vecs, ensemble_vecs).argsort(axis=1)[:,::-1][:, 1:10]

In [19]:
# for i, ns in enumerate(ensemble_nn):
#     print(i, df["제품명"].loc[i], [(n, df["제품명"][n]) for n in ns[:5]])
#     print("--------------------------------------------------------------------------------------------------------------------------------------------")
for i, ns in enumerate(base_nn):
    print(i, [n for n in ns[:6]])
    print("--------------------------------------------------------------------------------------------------------------------------------------------")

0 [30, 70, 139, 5, 56, 115]
--------------------------------------------------------------------------------------------------------------------------------------------
1 [44, 91, 105, 133, 18, 121]
--------------------------------------------------------------------------------------------------------------------------------------------
2 [121, 126, 85, 110, 3, 24]
--------------------------------------------------------------------------------------------------------------------------------------------
3 [126, 128, 24, 27, 111, 16]
--------------------------------------------------------------------------------------------------------------------------------------------
4 [129, 45, 113, 72, 9, 121]
--------------------------------------------------------------------------------------------------------------------------------------------
5 [139, 47, 56, 51, 128, 6]
-------------------------------------------------------------------------------------------------------------------------